In [ ]:
import numpy as np
import pandas as pd
import os
import re
import json
import joblib
import tensorflow as tf
from nltk.tokenize import word_tokenize
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Layer, Input, Embedding, Dense, Dropout, Concatenate
from tensorflow.keras.initializers import HeNormal
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import StratifiedShuffleSplit, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Specify output directory
output_dir = 'E:/Concrete_Freeze_Thaw_Damage_Prediction_Results/MSA_outputs'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Define a text preprocessing function
def preprocess_text(text):
    # Convert text to lowercase
    text = text.lower()
    # Remove only commas, periods, and colons
    text = re.sub(r'[,.:]', '', text)
    return text

# Read data from CSV
data_path = 'E:/NC/FTD-Data.csv'
data = pd.read_csv(data_path, encoding='gbk')

# Extract numerical features and target variables
X_numeric = data.iloc[:, :18].values.astype(np.float32)
y = data.iloc[:, -2:].values.astype(np.float32)
concrete_types = data['CT'].values  # Used for stratification

# Extract text features and preprocess
X_text = data.iloc[:, 18:22].astype(str)
X_text = X_text.applymap(preprocess_text)

# Tokenize text using NLTK's word_tokenize
tokenized_texts = X_text.applymap(word_tokenize)

# Create a vocabulary set and word index (starting from 1 since 0 is used for padding)
vocab = set([word for sublist in tokenized_texts.values.flatten() for word in sublist])
word_index = {word: i + 1 for i, word in enumerate(vocab)}
vocab_size = len(word_index)
print("Vocabulary size:", vocab_size)

# Save word index as CSV
vocab_df = pd.DataFrame(list(word_index.items()), columns=['Word', 'Index'])
vocab_df.to_csv(os.path.join(output_dir, 'vocab_index.csv'), index=False)

# Save word index as JSON
with open(os.path.join(output_dir, 'vocab_index.json'), 'w', encoding='utf-8') as json_file:
    json.dump(word_index, json_file, ensure_ascii=False, indent=4)

# Convert tokenized texts to sequences using the word index
sequences = tokenized_texts.applymap(lambda x: [word_index[word] for word in x])

# Determine the maximum sequence length for each text column and save it
max_lengths = [max(len(seq) for seq in sequences.iloc[:, i]) for i in range(4)]
max_lengths_df = pd.DataFrame({'Column': X_text.columns, 'Max_Length': max_lengths})
max_lengths_df.to_csv(os.path.join(output_dir, 'max_lengths.csv'), index=False)
print("Max lengths of each text column:", max_lengths)

# Split data using StratifiedShuffleSplit with a test size of 20%
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in sss.split(X_numeric, concrete_types):
    X_train_numeric, X_test_numeric = X_numeric[train_index], X_numeric[test_index]
    X_train_text, X_test_text = X_text.iloc[train_index], X_text.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    concrete_types_train, concrete_types_test = concrete_types[train_index], concrete_types[test_index]

# Convert text to sequences and apply padding per text column
X_train_text_padded = []
X_test_text_padded = []

for col in X_text.columns:
    train_texts = [word_tokenize(text) for text in X_train_text[col]]
    test_texts = [word_tokenize(text) for text in X_test_text[col]]
    train_sequences = [[word_index[word] for word in text if word in word_index] for text in train_texts]
    test_sequences = [[word_index[word] for word in text if word in word_index] for text in test_texts]
    X_train_text_padded.append(pad_sequences(train_sequences, maxlen=max_lengths[X_text.columns.get_loc(col)], padding='post'))
    X_test_text_padded.append(pad_sequences(test_sequences, maxlen=max_lengths[X_text.columns.get_loc(col)], padding='post'))

# Standardize numerical features
scaler = StandardScaler()
X_train_numeric_scaled = scaler.fit_transform(X_train_numeric)
X_test_numeric_scaled = scaler.transform(X_test_numeric)

# Save the scaler
scaler_path = os.path.join(output_dir, 'scaler.pkl')
joblib.dump(scaler, scaler_path)

# Print the shape of each padded text column (training set)
for column_name, padded_array in zip(X_text.columns, X_train_text_padded):
    print(f"Shape of {column_name}: {padded_array.shape}")

# Define the MultiHeadAttention layer
class MultiHeadAttention(Layer):
    def __init__(self, embed_dim, num_heads, proj_dim=None, output_dim=None, return_attention_weights=False, **kwargs):
        super(MultiHeadAttention, self).__init__(**kwargs)
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.proj_dim = proj_dim if proj_dim is not None else embed_dim // num_heads
        self.output_dim = output_dim if output_dim is not None else embed_dim
        self.return_attention_weights = return_attention_weights

        # Separate Dense layers for each head's Q, K, V projections
        self.q_dense_layers = [Dense(self.proj_dim, use_bias=False, name=f"query_projection_{i}") for i in range(num_heads)]
        self.k_dense_layers = [Dense(self.proj_dim, use_bias=False, name=f"key_projection_{i}") for i in range(num_heads)]
        self.v_dense_layers = [Dense(self.proj_dim, use_bias=False, name=f"value_projection_{i}") for i in range(num_heads)]

        # Output dense layer to combine all heads' outputs
        self.output_dense = Dense(self.output_dim, activation='relu', use_bias=False, name="output_dense")
        self.supports_masking = True  # Supports masking

    def call(self, inputs, mask=None):
        head_outputs = []
        attention_weights_list = []

        for i in range(self.num_heads):
            q = self.q_dense_layers[i](inputs)
            k = self.k_dense_layers[i](inputs)
            v = self.v_dense_layers[i](inputs)

            matmul_qk = tf.matmul(q, k, transpose_b=True)
            dk = tf.cast(tf.shape(k)[-1], tf.float32)
            scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)

            if mask is not None:
                mask = tf.cast(mask, tf.float32)
                mask_expanded = tf.expand_dims(mask, axis=-1)
                matmul_mask = tf.matmul(mask_expanded, mask_expanded, transpose_b=True)
                scaled_attention_logits += (1 - matmul_mask) * -1e9

            attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)
            attention_weights_list.append(attention_weights)
            attention_output = tf.matmul(attention_weights, v)
            head_outputs.append(attention_output)

        concat_heads = Concatenate(axis=-1)(head_outputs)
        output = self.output_dense(concat_heads)
        if self.return_attention_weights:
            return output, attention_weights_list
        return output

    def get_config(self):
        config = super().get_config()
        config.update({
            'embed_dim': self.embed_dim,
            'num_heads': self.num_heads,
            'proj_dim': self.proj_dim,
            'output_dim': self.output_dim,
            'return_attention_weights': self.return_attention_weights,
            'supports_masking': self.supports_masking
        })
        return config

# Define a function to create the model
def create_model(max_lengths, vocab_size, embedding_dim, num_heads, proj_dim, output_dim, include_attention_weights=False):
    inputs = []
    main_outputs = []
    attention_weights_outputs = []
    for i, max_len in enumerate(max_lengths):
        input_layer = Input(shape=(max_len,), dtype='int32', name=f"text_input_{i+1}")
        embedding_layer = Embedding(input_dim=vocab_size, output_dim=embedding_dim, mask_zero=True,
                                    trainable=True, name=f"embedding_{i+1}")(input_layer)
        # Apply one layer of multi-head attention
        attention_layer = MultiHeadAttention(embedding_dim, num_heads, proj_dim=proj_dim, output_dim=output_dim,
                                              return_attention_weights=include_attention_weights)
        if include_attention_weights:
            attention_output, attention_weights = attention_layer(embedding_layer)
            attention_weights_outputs.append(attention_weights)
        else:
            attention_output = attention_layer(embedding_layer)
        # Use a reduction operation (maximum) over the time axis
        attention_pooled_output = tf.reduce_max(attention_output, axis=1)
        main_outputs.append(attention_pooled_output)
        inputs.append(input_layer)
    # Combine text inputs with the numeric input
    numeric_input = Input(shape=(18,), dtype='float32', name="numeric_input")
    inputs.append(numeric_input)
    combined_features = Concatenate()([*main_outputs, numeric_input])
    # Fully connected layers
    x = Dense(128, activation='relu', kernel_initializer='he_normal')(combined_features)
    x = Dropout(0.2)(x)
    x = Dense(64, activation='relu', kernel_initializer='he_normal')(x)
    x = Dropout(0.2)(x)
    x = Dense(32, activation='relu', kernel_initializer='he_normal')(x)
    x = Dropout(0.2)(x)
    final_output = Dense(2)(x)
    # Compile the model
    model = Model(inputs=inputs, outputs=final_output)
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mae')
    return model

# Initialize dictionaries to store metrics and training history
metrics_dict = {
    'MAE': {0: [], 1: []},
    'MSE': {0: [], 1: []},
    'RMSE': {0: [], 1: []},
    'R2': {0: [], 1: []}
}
loss_history = []

# Initialize dictionaries for MAE per concrete type and counts (for training and validation)
concrete_type_mae = {ctype: [] for ctype in np.unique(concrete_types_train)}
concrete_type_counts_train = {ctype: [] for ctype in np.unique(concrete_types_train)}
concrete_type_counts_val = {ctype: [] for ctype in np.unique(concrete_types_train)}

# Set up 5-fold stratified cross-validation
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
for fold, (train_idx, val_idx) in enumerate(skf.split(X_train_numeric_scaled, concrete_types_train)):
    print(f"Training fold {fold+1}/5")

    # Split data for the current fold
    X_train_fold_numeric = X_train_numeric_scaled[train_idx]
    X_val_fold_numeric = X_train_numeric_scaled[val_idx]
    y_train_fold = y_train[train_idx]
    y_val_fold = y_train[val_idx]
    concrete_types_train_fold = concrete_types_train[train_idx]
    concrete_types_val_fold = concrete_types_train[val_idx]

    # Create a new model for the current fold
    final_model = create_model(max_lengths, vocab_size, embedding_dim=64, num_heads=2,
                               proj_dim=64, output_dim=32, include_attention_weights=False)

    # Prepare text inputs for training and validation
    train_inputs = [X_train_text_padded[i][train_idx] for i in range(len(max_lengths))]
    val_inputs = [X_train_text_padded[i][val_idx] for i in range(len(max_lengths))]
    train_inputs.append(X_train_fold_numeric)
    val_inputs.append(X_val_fold_numeric)

    # Train the model
    history = final_model.fit(
        train_inputs, y_train_fold,
        validation_data=(val_inputs, y_val_fold),
        epochs=3, batch_size=64, verbose=1
    )
    # Record training history
    loss_history.append(history.history)

    # Predict on validation data
    y_pred = final_model.predict(val_inputs)
    # Calculate MAE for each concrete type in the validation set
    for ctype in np.unique(concrete_types_train):
        ctype_mask = (concrete_types_val_fold == ctype)
        if np.any(ctype_mask):
            mae = mean_absolute_error(y_val_fold[ctype_mask], y_pred[ctype_mask])
            concrete_type_mae[ctype].append(mae)
            concrete_type_counts_train[ctype].append(np.sum(concrete_types_train_fold == ctype))
            concrete_type_counts_val[ctype].append(np.sum(ctype_mask))

    # Evaluate model performance for each target
    for target in range(2):
        mae = mean_absolute_error(y_val_fold[:, target], y_pred[:, target])
        mse_val = mean_squared_error(y_val_fold[:, target], y_pred[:, target])
        metrics_dict['MAE'][target].append(mae)
        metrics_dict['MSE'][target].append(mse_val)
        metrics_dict['RMSE'][target].append(np.sqrt(mse_val))
        metrics_dict['R2'][target].append(r2_score(y_val_fold[:, target], y_pred[:, target]))

    # Save the model for the current fold
    model_save_path = os.path.join(output_dir, f'model_fold_{fold+1}.h5')
    final_model.save(model_save_path)

# Save training history for each fold as CSV files
for i, history in enumerate(loss_history):
    df_history = pd.DataFrame(history)
    df_history.to_csv(os.path.join(output_dir, f'fold_{i+1}_history.csv'), index=False)

# Print evaluation metrics for each fold from cross-validation
for metric in metrics_dict:
    for target in metrics_dict[metric]:
        print(f"{metric} for target {target}: {metrics_dict[metric][target]}")
for fold in range(5):
    print(f"Metrics for fold {fold+1}:")
    for metric in ['MAE', 'MSE', 'RMSE', 'R2']:
        for target in range(2):
            print(f"  {metric} for target {target+1}: {metrics_dict[metric][target][fold]:.4f}")

# Evaluate the test set metrics using the saved models from each fold
test_metrics = {
    'MAE': {'target1': [], 'target2': []},
    'MSE': {'target1': [], 'target2': []},
    'RMSE': {'target1': [], 'target2': []},
    'R2': {'target1': [], 'target2': []}
}

for fold in range(1, 6):
    model_path = os.path.join(output_dir, f'model_fold_{fold}.h5')
    # Load the model for the current fold (make sure to use the same architecture)
    model = create_model(max_lengths, vocab_size, embedding_dim=64, num_heads=2,
                         proj_dim=64, output_dim=32, include_attention_weights=False)
    model.load_weights(model_path)
    
    # Prepare test inputs
    test_inputs = [X_test_text_padded[i] for i in range(len(max_lengths))]
    test_inputs.append(X_test_numeric_scaled)
    
    # Predict on test set
    predictions = model.predict(test_inputs)
    
    # Calculate performance metrics for each target and save predictions to CSV
    for target in range(2):
        mae = mean_absolute_error(y_test[:, target], predictions[:, target])
        mse = mean_squared_error(y_test[:, target], predictions[:, target])
        rmse = np.sqrt(mse)
        r2 = r2_score(y_test[:, target], predictions[:, target])
        
        test_metrics['MAE'][f'target{target+1}'].append(mae)
        test_metrics['MSE'][f'target{target+1}'].append(mse)
        test_metrics['RMSE'][f'target{target+1}'].append(rmse)
        test_metrics['R2'][f'target{target+1}'].append(r2)
        
        print(f"Fold {fold} - Target {target+1} Metrics:")
        print(f"  MAE: {mae:.4f}")
        print(f"  MSE: {mse:.4f}")
        print(f"  RMSE: {rmse:.4f}")
        print(f"  R2: {r2:.4f}")
        
        # Save predictions and corresponding actual values to CSV
        df = pd.DataFrame({
            'Category': concrete_types_test,
            'Actual Value': y_test[:, target],
            'Predicted Value': predictions[:, target],
            'Error': np.abs(y_test[:, target] - predictions[:, target])
        })
        df.to_csv(os.path.join(output_dir, f'Predictions_Fold_{fold}_Target_{target+1}.csv'), index=False)

# Calculate and print average test metrics for each target
average_test_metrics = {}
for metric in test_metrics:
    average_test_metrics[metric] = {}
    for target in test_metrics[metric]:
        average_value = np.mean(test_metrics[metric][target])
        average_test_metrics[metric][target] = average_value
        print(f"Average {metric} for {target}: {average_value:.4f}")
